# Get PASTIS modes

## --- LUVOIR A --

We will now perform a singular value decomposition (SVD) on the PASTIS matrix to get the PASTIS modes.

In [ ]:
# Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib inline
from astropy.io import fits
import astropy.units as u
import hcipy as hc
from hcipy.optics.segmented_mirror import SegmentedMirror

## Read the matrix

It's your choice wheter you read the analytical or numerical matrix.

In [ ]:
#matrix = fits.getdata('/Users/ilaginja/Documents/data_from_repos/pastis_data/2019-5-15_001_50nm/matrix_numerical/PASTISmatrix_num_HiCAT_piston_Noll1.fits')
matrix = fits.getdata('/Users/ilaginja/Documents/data_from_repos/pastis_data/2019-5-31_002_1nm/matrix_numerical/PASTISmatrix_num_piston_Noll1.fits')

plt.figure(figsize=(10, 10))
plt.imshow(matrix)
plt.colorbar()

## Get eigenmodes and eigenvalues

Lucie actually uses an SVD. I need to ask her why she doesn't just get the eigenmodes and eigenvalues directly, since we're dealing with a quadratic, symmetrical matrix.

In [ ]:
evals, evecs = np.linalg.eig(matrix)

In [ ]:
print('evals.shape: {}'.format(evals.shape))
print('evals:\n{}'.format(evals))

In [ ]:
print('evecs.shape: {}'.format(evecs.shape))
#print('evecs:\n{}'.format(evecs))

Which dimension are the eigenvectors in?

We know

$$M \cdot u = \lambda \cdot u$$

so we can test this.

In [ ]:
# Evaluate left and right side of eigenvalue problem for mode number n
n = 17

left = np.dot(matrix, evals[n])
right = np.dot(matrix, evecs[:, n])

# Compare them
print('Are the left and right side the same:')
print(np.allclose(left, right))

### Sort from lowest to highest eigenvalue

In [ ]:
# Sort them
sorted_evals = np.sort(evals)
sorted_indices = np.argsort(evals)
sorted_evecs = evecs[:, sorted_indices]

plt.figure(figsize=(14, 8))
#plt.plot(evals, label='Unsorted from eigendecomposition')
plt.plot(sorted_evals, label='Sorted lowest to highest evals')
#plt.semilogy()
plt.xlabel('Eigenmodes')
plt.ylabel('Eigenvalues')
plt.legend()
#plt.savefig('LUVOIR_eigenvals.pdf')

## Mode display

Instead of using the `SegmentedTelescopeAPLC` class to display the modes, which would rewuire us to load andn create all the files needed for the APLC propagation, we will simply use a `SegmentedMirror`.

In [ ]:
# Load aperture files needed for SM
nseg = 120
wvln = 638e-9

datadir = '/Users/ilaginja/Documents/LabWork/ultra/LUVOIR_delivery_May2019/'
aper_path = 'inputs/TelAp_LUVOIR_gap_pad01_bw_ovsamp04_N1000.fits'
aper_ind_path = 'inputs/TelAp_LUVOIR_gap_pad01_bw_ovsamp04_N1000_indexed.fits'
aper_read = hc.read_fits(os.path.join(datadir, aper_path))
aper_ind_read = hc.read_fits(os.path.join(datadir, aper_ind_path))

pupil_grid = hc.make_pupil_grid(dims=aper_ind_read.shape[0], diameter=15)
aper = hc.Field(aper_read.ravel(), pupil_grid)
aper_ind = hc.Field(aper_ind_read.ravel(), pupil_grid)

wf_aper = hc.Wavefront(aper, wvln)

# Load segment positions from fits header
hdr = fits.getheader(os.path.join(datadir, aper_ind_path))

poslist = []
for i in range(nseg):
    segname = 'SEG' + str(i+1)
    xin = hdr[segname + '_X']
    yin = hdr[segname + '_Y']
    poslist.append((xin, yin))
    
poslist = np.transpose(np.array(poslist))
seg_pos = hc.CartesianGrid(poslist)

In [ ]:
# Instantiate SM
sm = SegmentedMirror(aper_ind, seg_pos)

In [ ]:
mode = 77   # We start numbering at 0 here, 0-35 (Python nunmbering!)

sm.flatten()
for seg, val in enumerate(sorted_evecs[:, mode]):
    #print(val)
    sm.set_segment(seg+1, 0.000001*val/2, 0, 0)

# Propagate WF and display SM phase
wf_sm = sm(wf_aper)

hc.imshow_field(wf_sm.phase, cmap='RdBu')
plt.colorbar()

In [ ]:
# Lets compute all modes now for LUVOIR with a HCIPy SM
luvoir_modes = []

for mode in range(len(evals)):
    print('Working on mode {}/{}.'.format(mode+1, len(evals)))
    
    sm.flatten()
    for seg, val in enumerate(sorted_evecs[:, mode]):
        #print(val)
        sm.set_segment(seg+1, 0.000001*val/2, 0, 0)
        
    wf_sm = sm(wf_aper)
    luvoir_modes.append(wf_sm.phase)

In [ ]:
# Plot them all
pmin = -5e-7
pmax = 5e-7

plt.figure(figsize=(24, 20))
for mode in range(len(evals)):
    
    plt.subplot(12, 10, mode+1)
    hc.imshow_field(luvoir_modes[mode], cmap='RdBu')#, vmin=pmin, vmax=pmax)
    #plt.colorbar()
    plt.axis('off')
    plt.title('Mode ' + str(mode+1))
    
#plt.savefig('LUVOIR_modes_piston.pdf')